In [94]:
import sqlite3
import pandas as pd
import folium
from folium.plugins import FeatureGroupSubGr

ImportError: cannot import name 'FeatureGroupSubGr' from 'folium.plugins' (/home/apprenant/miniconda3/lib/python3.10/site-packages/folium/plugins/__init__.py)

In [ ]:
# Connexion à la base de données
conn = sqlite3.connect('bdd.db')

# Création du DataFrame objet
df_objet = pd.read_sql_query('SELECT * FROM objet', conn)
df_gare = pd.read_sql_query('SELECT * FROM gare', conn)
# Merge avec la table gare pour récupérer la latitude et longitude

df_objet = df_objet.merge(pd.read_sql_query('SELECT gare, lat, lon FROM gare', conn), on='gare')

df_gare['frequentation_2020']

0     55824824
1     32439307
2     92236287
3    114468385
4     21580625
5     13203603
6      2498330
Name: frequentation_2020, dtype: int64

In [ ]:
for gare in df_gare.gare.unique():
    print(gare)

Paris Gare de Lyon
Paris Montparnasse
Paris Gare du Nord
Paris Saint-Lazare
Paris Est
Paris Austerlitz
Paris Bercy


In [ ]:
annee = 2020
# frequentation = df_gare[f'frequentation_{annee}'].iloc[0]
# frequentation

df_gare['frequentation_2020'].iloc[0]

55824824

In [ ]:
import json

paris_map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Pour chaque gare, création de deux barplots
for gare in df_gare.gare.unique():
    # Nombre d'objets trouvés par type
    objets_par_type = df_objet.groupby('type')['id'].count()
    barplot_objets = folium.FeatureGroup(name="Objets par type")
    for type_objet, nb_objets in objets_par_type.items():
        folium.Marker(
            location=[df_gare['lat'].iloc[0], df_gare['lon'].iloc[0]],
            popup=f"{type_objet}: {nb_objets} objets trouvés",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(barplot_objets)

    # Fréquentation sur une année donnée
    annee = 2020  # Modifier pour l'année souhaitée
    frequentation = df_gare[f'frequentation_{annee}'].iloc[0]
    barplot_frequentation = folium.plugins.FeatureGroupSubGroup(
        paris_map, folium.plugins.MarkerCluster())
    folium.Marker(
        location=[df_gare['lat'].iloc[0], df_gare['lon'].iloc[0]],
        popup=f"Fréquentation en {annee}: {frequentation} voyageurs",
        icon=folium.Icon(color='green', icon='stats')
    ).add_to(barplot_frequentation)

    # Ajout des barplots à la carte folium
    barplot_objets.add_to(paris_map)
    barplot_frequentation.add_to(paris_map)

# Affichage de la carte folium
folium.LayerControl().add_to(paris_map)
paris_map._repr_html_ = lambda : json.dumps(paris_map.to_dict())
paris_map